In [1]:
import os

In [2]:
os.chdir("/Users/cyber/MLProjectWineQuality")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
%pwd

'/Users/cyber/MLProjectWineQuality'

In [5]:
from src.MLProject.constants import *
from src.MLProject.utils.common import read_yml, create_directories

In [6]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yml(config_filepath)
        self.params = read_yml(params_filepath)
        self.schema = read_yml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [7]:
%pwd

'/Users/cyber/MLProjectWineQuality'

In [8]:
import os
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [19]:
class DataIngestion:
    
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip[ file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-02-03 19:28:16,951: INFO: common: yml_file: config/config.yml loaded successfully]
[2025-02-03 19:28:17,000: INFO: common: yml_file: params.yml loaded successfully]
[2025-02-03 19:28:17,005: INFO: common: yml_file: schema.yml loaded successfully]
[2025-02-03 19:28:17,008: INFO: common: created directory at: artifacts]
[2025-02-03 19:28:17,010: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-03 19:28:17,016: INFO: 4271477071: File already exists of size: ~ 26 KB]
